**Satellite Derived Bathymetry (SDB) without sample elevation data by google earth engine**

**Accuracy dependent on your coefficient!!!**

Thepchai Srinoi, Phisan Santitamnont

Department of Survey Engineering

Chulalongkorn University

Additional reading for improve this code ...

https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/satellite_derived_bathymetry_mapping-sdbm/

Prepared your library ....

In [1]:
import os
import ee
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import folium as fo
import folium
import math as m
import math

Authenticate to GEE servers .... Generate Token first !!

In [2]:
# Authenticate to the Earth Engine servers
ee.Authenticate()
# Initialize the API
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=LSpxACFQIzoznjorAH9mHvL5cNeQMUjQQzge4jyId-4&tc=Y6xchVJYOiMAnUJPDmF1VdBAhlmAylFYXWAX7FGlv0o&cc=cfZXoWzHFevNKAKZT9cOpl5OXo42zdzQgVbSDoRyF4U

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiNVF5ujLRFwdWwPHX0Qoo9Opg5ETMhPQwAjxhvR5xnGaZlyj0tN1M

Successfully saved authorization token.


In [3]:
# Install Function for Cloud Mask
def maskS2clouds(image) :
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (qa.bitwiseAnd(cloudBitMask).eq(0)) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Map Visualization : Google Earth Engine
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

Select your area of interest, date, band 8 threshold for landmask

In [10]:
####################################### OVERVIEW SATELLITE IMAGE

lat_min, lat_max, lon_min, lon_max = (12.498140, 12.715250, 101.074208, 101.966847)
AOI = ee.Geometry.Polygon(
        [[[lon_min, lat_max],
           [lon_min, lat_min],
           [lon_max, lat_min],
           [lon_max, lat_max]]])

fromdate = '2021-1-1'
todate = '2022-1-1'

image = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(fromdate,todate).\
          filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',8).\
          filterBounds(AOI).map(maskS2clouds).mosaic().clip(AOI)

### LANDMASK BY BAND 8 THRESHOLD
B8_threshold = 0.04

B8_c = (image.select('B8').lt(B8_threshold)).selfMask()
B2_lm = image.select('B2').multiply(B8_c)
B3_lm = image.select('B3').multiply(B8_c)
B4_lm = image.select('B4').multiply(B8_c)
B8_lm = image.select('B8').multiply(B8_c)
image = image.addBands(B2_lm.rename('B2_lm'))
image = image.addBands(B3_lm.rename('B3_lm'))
image = image.addBands(B4_lm.rename('B4_lm'))
image = image.addBands(B4_lm.rename('B8_lm'))

############### VISUALIZATION ############################## 
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=11)

map_matched.add_ee_layer(image,{'min':0, 'max':0.3, 'bands':['B4','B3','B2']},'RGB_L2A')
map_matched.add_ee_layer(image,{'min':0, 'max':0.3, 'bands':['B4_lm','B3_lm','B2_lm']},'RGB_L2A_landmask')

display(map_matched.add_child(folium.LayerControl()))

Do the bathymetry calculation, please select your suitable coefficient ...

In [12]:
#### SATELLITE DERIVED BATHYMETRY ALGORITHM - STUMPF ALGORITHM

if 0:   # Open/Close SDB Algorithm
  A = (image.select('B2_lm')).multiply(1000)
  B = (image.select('B3_lm')).multiply(1000)
  B2B3 = (A.log()).divide(B.log())
  #SDB_ST = m0 + m1((ln1000B2)/(ln1000B3))
  image = image.addBands(B2B3.rename('B2B3'))

  # Use Coefficient from Koh Mark Trat : SDB Senior Project ...
  model_sdb_coef_ = 68.1061068420549
  model_sdb_intercept_ = -63.72996441

  SDB_ST = ((image.select('B2B3')).multiply(float(model_sdb_coef_))).add(model_sdb_intercept_)
  image = image.addBands(SDB_ST.rename('SDB_ST'))
  print('Stumpf finish ....')

############### VISUALIZATION ############################## 
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=11)

depth_palette = ['#FE1B09','#FEB009', '#FEFB09','#B8FE09', '#09FEF3', '#0927FE']

map_matched.add_ee_layer(image,{'min':0, 'max':0.3, 'bands':['B4','B3','B2']},'RGB_L2A')
map_matched.add_ee_layer(image.select('SDB_ST'),{'min': 0, 'max': 16, 'palette' : depth_palette},'SDB Stumpf') 

display(map_matched.add_child(folium.LayerControl()))

export to geotiff

In [ ]:
sdb_band = image.select('SDB_ST')
task = ee.batch.Export.image.toDrive(**{
    'image': sdb_band,
    'description': 'sdb_kohsamet',
    'folder':'mybathymetry',
    'scale': 10,
    'region': AOI.getInfo()['coordinates']
})
task.start()

import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: YMMIT4RHBAFINC5ESP7GNQXG).
Polling for task (id: YMMIT4RHBAFINC5ESP7GNQXG).
Polling for task (id: YMMIT4RHBAFINC5ESP7GNQXG).
Polling for task (id: YMMIT4RHBAFINC5ESP7GNQXG).
Polling for task (id: YMMIT4RHBAFINC5ESP7GNQXG).
Polling for task (id: YMMIT4RHBAFINC5ESP7GNQXG).
